# Лабораторная работа 4. Вариант 7. Группа 343

## Задание
Найти приближенное решение интегрального уравнения $u(x) - \int_{a}^{b}H(x,y)u(y)\,dy=f(x)$, используя одну из квадратурных формул:

1) Составная формула трапеций.

2) Составная формула средних прямоугольников.

3) Составная формула Симпсона.

4) Формулы Гаусса с 2, 3, 4 и т.д. узлами.

5) Составная формула Гаусса с двумя узлами.

6) Составная формула Гаусса с тремя узлами.

Количество разбиений в составных формулах рекомендуется удваивать до 
тех пор, пока значения приближенных решений в точках $a, (a+b)/2, b$ не 
будут совпадать с точностью $\varepsilon$.

In [1]:
import numpy as np
from numpy import linalg as LA
from tabulate import tabulate


A = 0
B = 1
N = 2
EPSILON = 0.001
PRECISION_DIGITS = 6


def find_solution_gauss(A, b, use_pivoting=False):
    n = len(A)
    if b.size != n:
        raise ValueError('Incompatible sizes of A and b')

    for i in range(n - 1):
        if use_pivoting:
            pivot_index = abs(A[i:, i]).argmax() + i
            if A[pivot_index, i] == 0:
                raise ValueError('A is singular')

            if pivot_index != i:
                A[[i, pivot_index]] = A[[pivot_index, i]]
                b[[i, pivot_index]] = b[[pivot_index, i]]
        else:
            if A[i, i] == 0:
                raise ValueError('Pivot is zero')

        for j in range(i + 1, n):
            ratio = A[j, i] / A[i, i]
            A[j, i:] -= ratio * A[i, i:]
            b[j] -= ratio * b[i]

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

    return x

### Уравнение
### $u(x) - 0.5\int_{0}^{1}th(xy)u(y)\,dy = x - 0.5$
### Метод
Составная формула Симпсона

In [2]:
def f(x):
    return x - 0.5


def H(x, y):
    return np.tanh(x * y).round(PRECISION_DIGITS)


def get_points(a, b, f, n, x):
    step = np.abs(b - a) / n
    points = []
    for i in range(n * 2 + 1):
        y = a + i * step
        f_xy = H(x, y)
        points.append((x, y, f_xy))
        
    return points


def find_simpson_coefs(points):
    n = len(points) // 2
    a = points[0][1]
    b = points[n][1]
    coefs = []
    k_out = (b - a) / (6 * n)
    
    coefs.append((k_out * points[0][2]).round(PRECISION_DIGITS))
    
    for i in range(1, len(points) - 1):
        k_in = 2 if i % 2 == 0 else 4
        coefs.append((k_out * k_in * points[i][2]).round(PRECISION_DIGITS))
        
    coefs.append((k_out * points[-1][2]).round(PRECISION_DIGITS))
    return coefs


def find_u_n(a, b, h, f, n, x, print_tmp_res=False):
    if print_tmp_res:
        print(f'Кол-во узлов: {n}')
    points = get_points(a, b, f, n, x)
    if print_tmp_res:
        print(f'Узлы (x, y, H(x, y)): {points}')
    simpson_coefs = find_simpson_coefs(points)
    
    step = np.abs(b - a) / n
    
    D = np.zeros((n, n))
    for j in range(n):
        xj = x + j * step
        for k in range(n):
            delta = 1 if j == k else 0
            ak = simpson_coefs[k]
            xk = x + k * step
            hjk = h(xj, xk)
            djk = delta - ak * hjk
            D[j, k] = djk
    
    g_values = []
    for i in range(0, n):
        x_i = A + i * step
        g_values.append(f(x_i))
        
    g = np.array([g_values]).T
    z = find_solution_gauss(np.copy(D), np.copy(g))
    if print_tmp_res:
        print(f'Коэф-ты квадратурной формулы: {z}')

    if print_tmp_res and n < 5:
        print(f'D: \n{D}')
        print(f'g: \n{g}\n\n\n')
    
    u_n = 0
    
    for i in range(n):
        u_n += simpson_coefs[i] * h(x, x + i * step) * z[i]
        
    u_n = 0.5 * u_n + f(x)

    return u_n

In [3]:
def find_max_diff(prev_values, values):
    return max([np.abs(values[i] - prev_values[i]) for i in range(len(values))])


def print_table(a, b, h, f, n, print_tmp_res=False):
    accurate = False
    rows = []
    u_n_a = find_u_n(a, b, h, f, n, a, print_tmp_res)
    u_n_ab = find_u_n(a, b, h, f, n, (a + b) / 2, print_tmp_res)
    u_n_b = find_u_n(a, b, h, f, n, b, print_tmp_res)
    
    rows.append([f'u^{n} (x)', str(u_n_a), str(u_n_ab), str(u_n_b)])
    if print_tmp_res:
            print(rows[-1])
    
    while not accurate:
        n *= 2
        new_u_n_a = find_u_n(a, b, h, f, n, a, print_tmp_res)
        new_u_n_ab = find_u_n(a, b, h, f, n, (a + b) / 2, print_tmp_res)
        new_u_n_b = find_u_n(a, b, h, f, n, b, print_tmp_res)
        rows.append([f'u^{n} (x)', str(new_u_n_a), str(new_u_n_ab), str(new_u_n_b)])
        error = find_max_diff([u_n_a, u_n_ab, u_n_b], [new_u_n_a, new_u_n_ab, new_u_n_b])
        if print_tmp_res:
            print(f'Точность: {error}')
        accurate = error < EPSILON
        if not accurate:
            (u_n_a, u_n_ab, u_n_b) = (new_u_n_a, new_u_n_ab, new_u_n_b)
    
    rows.append([f'u^{n} (x) - u^{n / 2} (x)', str(new_u_n_a - u_n_a), str(new_u_n_ab - u_n_ab), str(new_u_n_b - u_n_b)])
    rows.append(['Решение, полученное в 1-ом методе', rows[-2][1], rows[-2][2], rows[-2][3]])
    
    print(tabulate(rows, headers=['x', 'a', '(a+b)/2', 'b']))
    print(f'Оценка, полученная в 1-ом методе: {error}')


In [4]:
print_table(A, B, H, f, N, False)

x                                     a      (a+b)/2            b
---------------------------------  ----  -----------  -----------
u^2 (x)                            -0.5  0            0.5
u^4 (x)                            -0.5  0.00347212   0.509825
u^8 (x)                            -0.5  0.0051858    0.514783
u^16 (x)                           -0.5  0.00605155   0.517319
u^32 (x)                           -0.5  0.00648772   0.518606
u^64 (x)                           -0.5  0.00670679   0.519254
u^64 (x) - u^32.0 (x)               0    0.000219071  0.000647683
Решение, полученное в 1-ом методе  -0.5  0.00670679   0.519254
Оценка, полученная в 1-ом методе: 0.0006476831586021969


In [5]:
print_table(A, B, H, f, N, True)

Кол-во узлов: 2
Узлы (x, y, H(x, y)): [(0, 0.0, 0.0), (0, 0.5, 0.0), (0, 1.0, 0.0), (0, 1.5, 0.0), (0, 2.0, 0.0)]
Коэф-ты квадратурной формулы: [-0.5  0. ]
D: 
[[1. 0.]
 [0. 1.]]
g: 
[[-0.5]
 [ 0. ]]



Кол-во узлов: 2
Узлы (x, y, H(x, y)): [(0.5, 0.0, 0.0), (0.5, 0.5, 0.244919), (0.5, 1.0, 0.462117), (0.5, 1.5, 0.635149), (0.5, 2.0, 0.761594)]
Коэф-ты квадратурной формулы: [-0.5  0. ]
D: 
[[ 1.         -0.03772723]
 [ 0.          0.93782347]]
g: 
[[-0.5]
 [ 0. ]]



Кол-во узлов: 2
Узлы (x, y, H(x, y)): [(1, 0.0, 0.0), (1, 0.5, 0.462117), (1, 1.0, 0.761594), (1, 1.5, 0.905148), (1, 2.0, 0.964028)]
Коэф-ты квадратурной формулы: [-0.5  0. ]
D: 
[[ 1.         -0.13942809]
 [ 0.          0.84934585]]
g: 
[[-0.5]
 [ 0. ]]



['u^2 (x)', '-0.5', '0.0', '0.5']
Кол-во узлов: 4
Узлы (x, y, H(x, y)): [(0, 0.0, 0.0), (0, 0.25, 0.0), (0, 0.5, 0.0), (0, 0.75, 0.0), (0, 1.0, 0.0), (0, 1.25, 0.0), (0, 1.5, 0.0), (0, 1.75, 0.0), (0, 2.0, 0.0)]
Коэф-ты квадратурной формулы: [-0.5  -0.25  0.    0.25]
D

Коэф-ты квадратурной формулы: [-0.46149275 -0.44578043 -0.43007359 -0.41437189 -0.39867507 -0.38298283
 -0.36729494 -0.35161111 -0.33593113 -0.32025477 -0.3045818  -0.28891205
 -0.2732453  -0.25758139 -0.24192013 -0.22626137 -0.21060496 -0.19495076
 -0.17929863 -0.16364843 -0.14800005 -0.13235337 -0.1167083  -0.10106473
 -0.08542254 -0.06978167 -0.05414203 -0.03850353 -0.02286612 -0.00722969
  0.0084058   0.0240404   0.03967421  0.05530725  0.07093957  0.08657125
  0.10220231  0.11783279  0.13346274  0.14909221  0.1647212   0.18034978
  0.19597796  0.21160577  0.22723325  0.24286041  0.25848728  0.27411388
  0.28974024  0.30536636  0.32099228  0.33661801  0.35224356  0.36786894
  0.38349418  0.39911928  0.41474426  0.43036913  0.44599389  0.46161857
  0.47724315  0.49286765  0.5084921   0.52411648]
Точность: 0.0006476831586021969
x                                     a      (a+b)/2            b
---------------------------------  ----  -----------  -----------
u^2 (x)                   